See the setup instructions in `setup.md`.


https://github.com/ml-explore/mlx-examples


In [1]:
%%capture
%pip install --upgrade mlx-lm

In [2]:
%%capture
import mlx_lm
from IPython.display import display, clear_output, Markdown

In [ ]:
# Convert and/or quantize the model

# The original model will be downloaded to ~/.cache/huggingface/hub
hub_id = 'mlx-community/SmolLM2-1.7B-Instruct'
q_bits = 4

model_name = hub_id.split('/')[-1]
# The converted model will be saved to this directory
output_path = f'mlx-community/{model_name}-{q_bits}bit'
quantize = True

mlx_lm.convert(hf_path=hub_id, mlx_path=output_path, quantize=quantize, q_bits=q_bits)

In [ ]:
# Test the converted model

(model, tokenizer) = mlx_lm.load(output_path)

system_prompt = "You are a helpful assistant."
user_input = "1+1="

formatted_prompt = tokenizer.apply_chat_template(
    [
        # Not all models support the "system" role.
        # If it's not supported, you can prepend the system prompt to the first "user" message.
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
        # You can add more "assistant" and "user" turns to represent the chat history in subsequent interactions
    ],
    tokenize=False,
    add_generation_prompt=True,
)


def display_stream(generator):
  full_response = ""
  for token in generator:
    full_response += token
    clear_output(wait=True)
    display(Markdown(f"{full_response}"))
  clear_output(wait=True)
  return full_response


response_generator = mlx_lm.stream_generate(model, tokenizer, prompt=formatted_prompt, max_tokens=1000)
response = display_stream(response_generator)
display(Markdown(f"{response}"))